# 04_07: SOLUTION - Evaluating your GraphRAG Pipeline

In [ ]:
import os
from dotenv import load_dotenv

from langchain.evaluation.qa.eval_chain import QAEvalChain
from langchain_neo4j import GraphCypherQAChain, Neo4jGraph
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate

In [ ]:
load_dotenv(dotenv_path='../.env')

URI = os.getenv("NEO4J_URI")
USER = os.getenv("NEO4J_USER")
PWD = os.getenv("NEO4J_PWD")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [ ]:
enhanced_graph = Neo4jGraph(url=URI, username=USER, password=PWD, enhanced_schema=True)
llm = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model="gpt-4o", temperature=0)
eval_chain = QAEvalChain.from_llm(llm)

In [ ]:
examples = [
    {"query": "What sports is the International Ski And Snowboard Federation responsible for?",
     "answer": "Alpine Skiing, Freestyle Skiing, Snowboarding, Nordic Combined, Ski Jumping, Cross-Country Skiing"},
    {"query": "What activity are ski poles not used in?", "answer": "Ski jumping"},
    {"query": "Who do athletes get help from?", "answer": "Coaches, Peer Mentors, and Sports Psychologists"},
]

In [ ]:
graph_chain = GraphCypherQAChain.from_llm(
    graph=enhanced_graph,
    llm=llm,
    verbose=True,
    allow_dangerous_requests=True,
)

In [ ]:
def evaluate_graph_rag(graph_chain, examples):

    # Generate predictions by querying the graph
    predictions = []
    for ex in examples:
        graph_response = graph_chain.invoke({"query": ex["query"]})
        predictions.append({"result": graph_response["result"].strip()})

    # Run evaluation
    eval_chain = QAEvalChain.from_llm(llm)
    results = eval_chain.evaluate(examples, predictions)

    # Print output
    correct = 0
    for i, res in enumerate(results):
        print(f"Query: {examples[i]['query']}")
        print(f"Prediction from graph: {predictions[i]['result']}")
        print(f"Gold answer: {examples[i]['answer']}")
        print(f"Grade: {res['results']}")
        print("---")
        if res["results"] == "CORRECT":
            correct += 1

    accuracy = correct / len(examples)
    print(f"Graph QA Accuracy: {accuracy:.2f}")

In [ ]:
evaluate_graph_rag(graph_chain, examples)

In [ ]:
examples = [
    {"query": "What activity is the Arlberg-Kandahar Races associated with?",
     "answer": "The Arlberg-Kandahar Races are associated with Ski Touring."},
    {"query": "How does research play a role?",
     "answer": "Research improves competition by looking at Surface Science and Friction Reduction and has implications for Competitive Performance and Recreational Enjoyment"},
    {"query": "What techniques does the Skiing Community use to enhance Safety?",
     "answer": "The Skiing Community innovates on Techniques that enhance Safety."},
]

In [ ]:
evaluate_graph_rag(graph_chain, examples)

In [ ]:
CYPHER_GENERATION_TEMPLATE = """

You are an expert Neo4j developer. Your job is to generate a Cypher query that will traverse exactly 1–3 hops out from your focus node.

Steps:

1. Identify the main entity in the question and Capitalize Multi-Word IDs
2. Write a variable-length MATCH using the new syntax:
   MATCH path = (a {{id: "{{entity}}"}})-[r]-{{1,3}}(b)
   RETURN path;

Only return the Cypher query.

Schema:
{schema}

Question:
{question}
"""

cypher_generation_prompt = PromptTemplate(
    template=CYPHER_GENERATION_TEMPLATE,
    input_variables=["schema", "question"],
)

enhanced_graph_chain = GraphCypherQAChain.from_llm(
    graph=enhanced_graph,
    llm=llm,
    cypher_prompt=cypher_generation_prompt,
    verbose=True,
    allow_dangerous_requests=True,
)

In [ ]:
evaluate_graph_rag(enhanced_graph_chain, examples)

In [ ]:
ANSWER_SYNTHESIS_TEMPLATE = """
You have a list of variable‑length paths from Neo4j in the form:

(path = [(a)-[:REL1]->(X)-[:REL2]->(b), …])

Your job: extract the intermediate node X and the two relationship names REL1 and REL2,
then produce one sentence of the form:

“The {a.id} {rel1_verb} on {X.id} that {rel2_verb} {b.id}.”

Example:

Paths:
1. (Skiing Community)-[INNOVATE]-(Techniques)-[ENHANCE]-(Safety)

Answer:
The Skiing Community innovates on Techniques that enhance Safety.

Now, given:
{paths}

Answer:
"""

answer_prompt = PromptTemplate(
    template=ANSWER_SYNTHESIS_TEMPLATE,
    input_variables=["paths"],
)

full_graph_chain = GraphCypherQAChain.from_llm(
    llm=llm,
    graph=enhanced_graph,
    cypher_prompt=cypher_generation_prompt,
    answer_prompt=answer_prompt,      # ← override the summarizer
    verbose=True,
    allow_dangerous_requests=True,
)

In [ ]:
evaluate_graph_rag(full_graph_chain, examples)